In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

from bs4 import BeautifulSoup
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from wordcloud import WordCloud, STOPWORDS
from nltk.tokenize import word_tokenize

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
#Criando conjunto de "Dados Estruturados"

# Importando Dados para o COLAB para fazer a montagem do drive no notebook
from google.colab import drive
drive.mount('/content/drive')

print('Importação completa!')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Importação completa!


In [ ]:
#Notícias Verdadeiras Drive PET
X = []
y = []

N = 3600 # Número de notícias coletadas do banco de "notícias genuínas"

i = 0
while i < N:
    try:
        file = open("/content/drive/MyDrive/Projetos/Big PET - Data Science/Detector de FakeNews/dataset/true/"+str(i+1)+".txt")
        # Os arquivos contidos na pasta apresentam como identidade um número mais a extensão .txt
        X.append(file.readlines())
        y.append(True)
        file.close()
    except:
        print('Erro na leitura ou ausência do arquivo: ', i)
        # Sequencialmente, cada arquivos será checado, um a um
        X.append([None])
        y.append([None])
    i += 1

news = pd.DataFrame(data = list(zip(X, y)), columns = ['Conteúdo', 'Veredicto'])
news.index = np.arange(len(news))     
print('Validação completa')

In [ ]:
#Notícias Verdadeiras Drive Leonardo
X = []
y = []

N = 3600 # Número de notícias coletadas do banco de "notícias genuínas"

i = 0
while i < N:
    try:
        file = open("/content/drive/MyDrive/PET/Detector de FakeNews/dataset/true/"+str(i+1)+".txt")
        # Os arquivos contidos na pasta apresentam como identidade um número mais a extensão .txt
        X.append(file.readlines())
        y.append(True)
        file.close()
    except:
        print('Erro na leitura ou ausência do arquivo: ', i)
        # Sequencialmente, cada arquivos será checado, um a um
        X.append([None])
        y.append([None])
    i += 1

news = pd.DataFrame(data = list(zip(X, y)), columns = ['Conteúdo', 'Veredicto'])
news.index = np.arange(len(news))     
print('Validação completa')

Erro na leitura ou ausência do arquivo:  585
Erro na leitura ou ausência do arquivo:  1606
Validação completa


In [ ]:
#Tratando os Dados
#Subsituição de caracteres invalidos das noticias verdadeiras 
news_np = pd.DataFrame.to_numpy(news['Conteúdo'])
lista = []
for i in range(len(news_np)):
  lista.append(news_np[i])
  if lista[i][0] != None:
    lista[i][0] = lista[i][0].replace('\n', '') #Removendo/alterando caracteres
    lista[i][0] = lista[i][0].replace('é', 'e')
    lista[i][0] = lista[i][0].replace('ê', 'e')
    lista[i][0] = lista[i][0].replace('á', 'a')
    lista[i][0] = lista[i][0].replace('â', 'a')
    lista[i][0] = lista[i][0].replace('à', 'a')
    lista[i][0] = lista[i][0].replace('ã', 'a')
    lista[i][0] = lista[i][0].replace('õ', 'o')
    lista[i][0] = lista[i][0].replace('ó', 'o')
    lista[i][0] = lista[i][0].replace('ô', 'o')
    lista[i][0] = lista[i][0].replace('ú', 'u')
    lista[i][0] = lista[i][0].replace('í', 'i')
    lista[i][0] = lista[i][0].replace('ç', 'c') 
  lista[i] = lista[i][0]
news['Conteúdo'] = lista


In [ ]:
# Dataset de notícias verdadeiras

news.head(10)



,Conteúdo,Veredicto
0,O Podemos decidiu expulsar o deputado federal...,True
1,Em evento realizado nesta terca-feira para div...,True
2,﻿Apos o prefeito de Manaus Arthur Virgilio (PS...,True
3,﻿Doria vai receber Ze Celso apos reuniao com r...,True
4,"Gustavo Pedreira Ferraz, que admitiu buscar ma...",True
5,"Muito antes da Operacao Lava Jato existir, em ...",True
6,Ministerio Publico do Maranhao (MP-MA) ingress...,True
7,Cuba comemorara o primeiro aniversario da mort...,True
8,s doleiros acusados de lavar dinheiro roubado ...,True
9,"Bolsonaro e um liberal completo, diz president...",True


In [ ]:
#Notícias Falsas
X = []
y = []

N = 3600  # Número de notícias coletadas do banco de "notícias falsas"
i = 0
while i < N:
    try:
        file = open("/content/drive/MyDrive/PET/Detector de FakeNews/dataset/fake/"+str(i+1)+".txt")
        # Os arquivos contidos na pasta apresentam como identidade um número mais a extensão .txt
        X.append(file.readlines())
        y.append(False)
        file.close()
    except:
        print('Erro na leitura ou ausência do arquivo: ', i)
        # Sequencialmente, cada arquivos será checado, um a um
        X.append([None])
        y.append([None])
    i += 1
fakenews = pd.DataFrame(data = list(zip(X, y)), columns = ['Conteúdo', 'Veredicto'])
fakenews.index = np.arange(len(fakenews)) + len(news)
print('Validação completa')

Erro na leitura ou ausência do arquivo:  585
Erro na leitura ou ausência do arquivo:  1606


In [ ]:
#Tratando os Dados
#Subsituição de caracteres invalidos das noticias falsas
news_np = pd.DataFrame.to_numpy(fakenews['Conteúdo'])
lista = []
for i in range(len(news_np)):
  lista.append(news_np[i])
  if lista[i][0] != None:
    lista[i][0] = lista[i][0].replace('\n', '') #Removendo/alterando caracteres
    lista[i][0] = lista[i][0].replace('é', 'e')
    lista[i][0] = lista[i][0].replace('ê', 'e')
    lista[i][0] = lista[i][0].replace('á', 'a')
    lista[i][0] = lista[i][0].replace('â', 'a')
    lista[i][0] = lista[i][0].replace('à', 'a')
    lista[i][0] = lista[i][0].replace('ã', 'a')
    lista[i][0] = lista[i][0].replace('õ', 'o')
    lista[i][0] = lista[i][0].replace('ó', 'o')
    lista[i][0] = lista[i][0].replace('ô', 'o')
    lista[i][0] = lista[i][0].replace('ú', 'u')
    lista[i][0] = lista[i][0].replace('í', 'i')
    lista[i][0] = lista[i][0].replace('ç', 'c') 
  lista[i] = lista[i][0]
fakenews['Conteúdo'] = lista


In [ ]:
# Dataset de notícias falsas

fakenews.head(10)

In [ ]:
# Removendo os poucos valores inválidos
news = news.dropna() 
fakenews = fakenews.dropna()

In [ ]:
news.describe()

In [ ]:
news.count()

In [ ]:
fakenews.describe()

In [ ]:
fakenews.count()

In [ ]:
# Unindo conjunto fakenews e news
df = pd.concat([news, fakenews], ignore_index = True) 
df.index = np.arange(len(df))

In [ ]:
# Importando palavras mais comuns do Português

file = open('/content/drive/MyDrive/Detector de FakeNews/stopwords.txt')
stopwords = file.readlines()
for i in range(len(stopwords)):
  stopwords[i] = stopwords[i].replace(' ', '')
  stopwords[i] = stopwords[i].replace('\n', '')
  stopwords[i] = stopwords[i].replace('é', 'e')
  stopwords[i] = stopwords[i].replace('ê', 'e')
  stopwords[i] = stopwords[i].replace('á', 'a')
  stopwords[i] = stopwords[i].replace('â', 'a')
  stopwords[i] = stopwords[i].replace('à', 'a')
  stopwords[i] = stopwords[i].replace('ã', 'a')
  stopwords[i] = stopwords[i].replace('õ', 'o')
  stopwords[i] = stopwords[i].replace('ô', 'o')
  stopwords[i] = stopwords[i].replace('ó', 'o')
  stopwords[i] = stopwords[i].replace('ú', 'u')
  stopwords[i] = stopwords[i].replace('í', 'i')
  stopwords[i] = stopwords[i].replace('ç', 'c') 
file.close()


In [ ]:
stopwords = stopwords[:50] 

In [ ]:
#definindo função que remove as stopwords do texto
stop = stopwords
def remove_stopwords(text):
    final_text = []
    text = text.lower()
    for i in text.split():
        if i.strip() not in stop:
            final_text.append(i.strip())
    return " ".join(final_text)

#aplicaando a função remove_stopwords
df['Conteúdo']=df['Conteúdo'].apply(remove_stopwords)

In [ ]:
#lemmatization
# Init the Wordnet Lemmatizer
lemmatizer = WordNetLemmatizer()

#A function which takes a sentence/corpus and gets its lemmatized version.
def lemmatize_text(text):
    token_words=word_tokenize(text) 
#we need to tokenize the sentence or else lemmatizing will return the entire sentence as is.
    lemma_sentence=[]
    for word in token_words:
        lemma_sentence.append(lemmatizer.lemmatize(word))
        lemma_sentence.append(" ")
    return "".join(lemma_sentence)

#Apply function on text column
df['Conteúdo']=df['Conteúdo'].apply(lemmatize_text)
df

In [ ]:
#word cloud para as fake news
cloud = WordCloud(max_words = 500, stopwords = STOPWORDS, background_color = "white").generate(" ".join(df[df.Veredicto == True].Conteúdo))
plt.figure(figsize=(10, 10))
plt.imshow(cloud, interpolation="bilinear")
plt.axis("off")
plt.tight_layout(pad=0)
plt.show()

In [ ]:
cloud = WordCloud(max_words = 500, stopwords = STOPWORDS, background_color = "white").generate(" ".join(df[df.Veredicto == False].Conteúdo))
plt.figure(figsize=(10, 10))
plt.imshow(cloud, interpolation="bilinear")
plt.axis("off")
plt.tight_layout(pad=0)
plt.show()

In [ ]:
#finding n-grams
texts = ''.join(str(df['Conteúdo'].tolist()))

# first get individual words
tokenized = texts.split()

In [ ]:
#unigram
unigram = (pd.Series(nltk.ngrams(tokenized, 1)).value_counts())[:20]
unigram.sort_values().plot.barh(width=.9, figsize=(12, 8))
plt.title('20 Most Frequently Occuring Unigrams')
plt.ylabel('Unigram')
plt.xlabel('# of Occurances')

In [ ]:
#bigrams
bigram = (pd.Series(nltk.ngrams(tokenized, 2)).value_counts())[:20]
bigram.sort_values().plot.barh(width=.9, figsize=(12, 8))
plt.title('20 Most Frequently Occuring Bigrams')
plt.ylabel('Bigram')
plt.xlabel('# of Occurances')

In [ ]:
#modeling
def get_prediction(vectorizer, classifier, X_train, X_test, y_train, y_test):
    pipe = Pipeline([('vector', vectorizer),
                    ('model', classifier)])
    model = pipe.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print("Accuarcy: {}".format(round(accuracy_score(y_test, y_pred)*100,2)))
    cm = confusion_matrix(y_test, y_pred)
    print("Confusion Matrix: \n", cm)


In [ ]:
#pipeline implementation
Conteudo = df['Conteúdo']
Conteudo = Conteudo.astype(str)

Veredicto = df['Veredicto']
Veredicto = Veredicto.astype(bool)

X_train, X_test, y_train, y_test = train_test_split(Conteudo, Veredicto, test_size = 0.3, random_state= 0)
classifiers = [LogisticRegression(),KNeighborsClassifier(n_neighbors=5), DecisionTreeClassifier(), GradientBoostingClassifier(),RandomForestClassifier()]


In [ ]:
Conteudo

In [ ]:
for classifier in classifiers:
    get_prediction(TfidfVectorizer(), classifier, X_train, X_test, y_train, y_test)

In [ ]:
dataBase = pd.DataFrame(data = list(zip(Conteudo, Veredicto)), columns = ['Conteúdo', 'Veredicto'])
dataBase